In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adamax
from keras.wrappers.scikit_learn import KerasRegressor
from keras import backend as K

### 데이터 생성

In [2]:
# train = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/dataset/spmm-latency-dataset/extract-dataset-using-d-optimal/dataset/nonsquare-train-1035-from-spmm-contain-todense-over-3s-1293.csv')
# test = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/dataset/spmm-latency-dataset/extract-dataset-using-d-optimal/dataset/compare-recommendation-method-performance.csv')

In [3]:
# train = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/dataset/spmm-dataset/dataset/nonsquare-train-1035-from-spmm-contain-todense-over-3s-1293.csv')
# test = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/dataset/spmm-dataset/dataset/nonsquare-test-258-from-spmm-contain-todense-over-3s-1293.csv')

In [4]:
train = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/dataset/spmm-dataset/dataset/nonsquare-train-1035-from-spmm-contain-todense-over-3s-1293.csv')
test = pd.read_csv('/Users/bdlab/Downloads/block-matrix-multiply-tracking.csv')

In [ ]:
# # Train
# X_train = train[['lr','lc','rc','ld','rd','lnnz','rnnz']] 
# sp_smdm_y_train = train['sp_smdm']
# bz_smsm_y_train = train['bz_smsm']

# # Test
# X_test = test[['lr','lc','rc','ld','rd','lnnz','rnnz']] 
# sp_smdm_y_test = test['sp_smdm']
# bz_smsm_y_test = test['bz_smsm']

In [5]:
# Train
X_train = train[['lr','lc','rc','ld','rd','lnnz','rnnz']] 
sp_smdm_y_train = train['sp_smdm']
#bz_smsm_y_train = train['bz_smsm']

# Test
X_test = test[['lr','lc','rc','ld','rd','lnnz','rnnz']] 
sp_smdm_y_test = test['smdm_ground_thruth']
#bz_smsm_y_test = test['bz_smsm']

### 데이터 전처리

In [6]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 객체 생성
minmax_scaler = MinMaxScaler()

# 훈련데이터의 모수 분포 저장
minmax_scaler.fit(X_train)

# 훈련 데이터 스케일링
X_train = minmax_scaler.transform(X_train)

# 테스트 데이터 스케일링
X_test = minmax_scaler.transform(X_test)

### Metric 함수 생성

In [7]:
# RMSE
def rmse(y_true, y_pred):
    rmse = K.sqrt(K.mean(K.square(y_pred - y_true))) 
    return rmse

In [8]:
# MAPE
def mean_absolute_percentage_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

### 기본 모델 구조

In [9]:
def create_model(dense_nparams, dense_layer_sizes , input_optimizer, input_kernel_initializer, input_dropout, input_lr):

    model=Sequential()
    model.add(Dense(dense_nparams, activation="relu", input_shape=(X_train.shape[1],), kernel_initializer=input_kernel_initializer))  
    model.add(Dropout(input_dropout),)
    
    # dense_layer_sizes 만큼 layer 추가
    for layer_size in dense_layer_sizes:
        model.add(Dense(layer_size, activation='relu', kernel_initializer=input_kernel_initializer))
        model.add(Dropout(input_dropout), )
    
    model.add(Dense(1))

    optimizer = input_optimizer(lr=input_lr)
    
    model.compile(optimizer = optimizer ,
                  loss='mape',
                  metrics=['mape'])
    return model

### sp_smdm dnn 모델

In [10]:
# 모델 정의
sp_smdm_model = create_model(512,
                            (128, 64, 16, 8),
                            Adagrad,
                            'he_normal',
                            0,
                            0.09)

# 에포크가 끝날 때마다 점(.)을 출력해 훈련 진행 과정을 표시합니다
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        #print("epoch : {}, logs : {}".format(epoch,logs))
        print('.', end='')

# monitor는 어떤 매개변수를 볼 것인지 입니다.
# patience 매개변수는 성능 향상을 체크할 에포크 횟수입니다
# 지정된 에포크 횟수 동안 성능 향상이 없으면 자동으로 훈련이 멈춥니다.
early_stop = keras.callbacks.EarlyStopping(monitor='val_mape', patience=100)

EPOCHS = 1000

# 훈련 정확도와 검증 정확도 출력
# 에포크마다 훈련 상태를 점검하기 위해 EarlyStopping 콜백(callback)을 사용합니다.
history = sp_smdm_model.fit(X_train, 
                    sp_smdm_y_train,
                    epochs=EPOCHS, 
                    validation_split = 0.1, 
                    verbose =0, 
                    callbacks=[early_stop, PrintDot()])



....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
......

### bz_smsm dnn 모델

In [20]:
# 모델 정의
bz_smsm_model = create_model(1024,
                            (128, 64, 32, 16),
                            Adagrad,
                            'normal',
                            0,
                            0.07)

# 에포크가 끝날 때마다 점(.)을 출력해 훈련 진행 과정을 표시합니다
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        #print("epoch : {}, logs : {}".format(epoch,logs))
        print('.', end='')

# monitor는 어떤 매개변수를 볼 것인지 입니다.
# patience 매개변수는 성능 향상을 체크할 에포크 횟수입니다
# 지정된 에포크 횟수 동안 성능 향상이 없으면 자동으로 훈련이 멈춥니다.
early_stop = keras.callbacks.EarlyStopping(monitor='val_mape', patience=100)

EPOCHS = 1000

# 훈련 정확도와 검증 정확도 출력
# 에포크마다 훈련 상태를 점검하기 위해 EarlyStopping 콜백(callback)을 사용합니다.
history = bz_smsm_model.fit(X_train, 
                    bz_smsm_y_train,
                    epochs=EPOCHS, 
                    validation_split = 0.1, 
                    verbose =0, 
                    callbacks=[early_stop, PrintDot()])



....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
.............

### 예측 성능

In [11]:
# sp_smdm 테스트데이터 예측
sp_smdm_y_pred = sp_smdm_model.predict(X_test).reshape(-1,)
print("sp_smdm 테스트데이터 예측 mape : {}\n".format(mean_absolute_percentage_error(sp_smdm_y_test,sp_smdm_y_pred)))

# # bz_smsm 테스트데이터 예측
# bz_smsm_y_pred = bz_smsm_model.predict(X_test).reshape(-1,)
# print("bz_smsm 테스트데이터 예측 mape : {}\n".format(mean_absolute_percentage_error(bz_smsm_y_test,bz_smsm_y_pred)))

sp_smdm 테스트데이터 예측 mape : 3486.239285637185



### 예측한, sp_smdm_y_pred 와 bz_smsm_y_pred 중 작은 값으로 y_pred_label 생성

In [41]:
y_pred_label = []

for i in range(len(X_test)):
    latency_list = []
    latency_list.append(sp_smdm_y_pred[i])
    latency_list.append(bz_smsm_y_pred[i])
    y_pred_label.append(latency_list.index(min(latency_list)))

### 예측한, sp_smdm_y_pred 와 bz_smsm_y_pred 중 작은 값으로 y_pred_latency 생성

In [42]:
y_pred_latency = []

for i in range(len(X_test)):
    latency_list = []
    latency_list.append(sp_smdm_y_pred[i])
    latency_list.append(bz_smsm_y_pred[i])
    y_pred_latency.append(min(latency_list))

### 필요시, y_pred_latency 를 csv 에 추가

In [13]:
# test['proposed_system_latency'] = list(map(int, y_pred_latency))
# test.to_csv('/Users/bdlab/Downloads/block-matrix-multiply-tracking-contain-pred-latency.csv', index=False)  
test['smdm_model_predict'] = list(map(int, sp_smdm_y_pred))
test.to_csv('/Users/bdlab/Downloads/block-matrix-multiply-tracking-contain-pred-latency.csv', index=False)  

### 실제값인, sp_smdm_y_test 과 bz_smsm_y_test 을 통해, y_real_label 생성

In [44]:
y_real_label = []

for i in range(len(X_test)):
    latency_list = []
    latency_list.append(test.loc[i,'sp_smdm'])
    latency_list.append(test.loc[i,'bz_smsm'])
    y_real_label.append(latency_list.index(min(latency_list)))

### y_pred_label 과 y_real_label 간 accuracy 측정

In [45]:
accuracy_score(y_pred_label, y_real_label)

0.9224806201550387

### 결과 dataframe 생성

In [27]:
# pd.options.display.float_format = '{:.5f}'.format
# temp = test[['lr','lc','rc','ld','rd','lnnz','rnnz','sp_smdm','bz_smsm']]
# pd.concat([temp,pd.DataFrame(y_pred_label,columns=['y_pred_label']),pd.DataFrame(sp_smdm_y_pred,columns=['sp_smdm_y_pred']),pd.DataFrame(bz_smsm_y_pred,columns=['bz_smsm_y_pred']), pd.DataFrame(y_real_label,columns=['y_real_label']) ],axis=1)

### 모델 저장

In [54]:
sp_smdm_model.save('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/model-save/dnn_sp_smdm_model')
bz_smsm_model.save('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/model-save/dnn_bz_smsm_model')

INFO:tensorflow:Assets written to: /Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/model-save/dnn_sp_smdm_model/assets
INFO:tensorflow:Assets written to: /Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/model-save/dnn_bz_smsm_model/assets


In [56]:
import tensorflow as tf

sp_smdm_load = tf.keras.models.load_model('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/model-save/dnn_sp_smdm_model') 


sp_smdm_y_pred_test = sp_smdm_load.predict(X_test).reshape(-1,)
print("sp_smdm 테스트데이터 예측 mape : {}\n".format(mean_absolute_percentage_error(sp_smdm_y_test,sp_smdm_y_pred_test)))

sp_smdm 테스트데이터 예측 mape : 10.985893010694427

